In [1]:
import os
import sys
import scipy.io as spio
import numpy as np
import plotly.graph_objects as pgo
import re

sys.path.append("../")
from helpers.get_density import get_density
from helpers.get_spins import get_spins
from helpers.downscale import downscale_3d
from plot_multiple_3Ds import plot_multiple_3Ds
from loader import get_snap_paths, load_Psi
from helpers.grids import get_kw_square_nonzero_grid, get_radius_grid, get_space_grid
from multiprocessing import Pool

In [2]:
box_N = 96
box_L = 400.0
box_dx = box_L / box_N

In [4]:
lambdas = [0]
runs = [1]
dtos = [1, 2, 4]
to_loads = []
for ld in lambdas:
	for run in runs:
		sr = {}
		for dto in dtos:
			sim_path = f"../out_remote/2022-09-11/4-solitons-attractive,lambda={ld},run={run},dto={dto}/"
			snap_paths = get_snap_paths(sim_path, box_L, box_N)
			for (itr, time, path) in snap_paths:
				to_loads.append((ld, run, dto, itr, time, path))


In [ ]:
def load_one(x):
	(ld, run, dto, itr, time, path) = x
	return (ld, run, dto, itr, time, load_Psi(path))

with Pool(32) as p:
 	loadeds = p.map(load_one, to_loads)

In [28]:

def test_for(test_ld, test_run):
	snaps = {dto: [] for dto in dtos}
	for (ld, run, dto, itr, time, Psi3) in sorted(loadeds):
		if (test_ld == ld and test_run == run):
			snaps[dto].append((time, Psi3))
	
	rhos = {dto: [(t,get_density(Psi3)) for (t,Psi3) in Psi3s] for (dto, Psi3s) in snaps.items()}
	dt1 = (2 / np.pi) * (box_dx)**2
	for ((t1, rho1), (t2, rho2), (t4, rho4)) in zip(rhos[1], rhos[2], rhos[4]):
		assert(abs(t1-t2) < 20 and abs(t2-t4) < 20)
		v = np.abs(rho1 - rho2) / np.abs(rho2 - rho4)
		q = np.log2(v)
		# print(np.percentile(q, 1), np.percentile(q, 10), np.percentile(q, 50), np.percentile(q, 90), np.percentile(q, 99))
		filt = (rho1 > 1E-5) + (rho2 > 1E-5) + (rho4 > 1E-5)
		print(len(q[filt]))
		print(np.average(q))
		print(np.average(q[filt]))
		
test_for(0, 1)

5388
1.6330975249729194
2.0347625708196326
5515
1.3106160641610194
1.1862418653851317
5506
1.243665927566465
1.014223576414701
5144
1.1812393592189945
0.9726910369592949
6258
1.20948280941929
1.1998539358264586
5659
1.7124344096294581
1.8524189696923472
4532
1.9579610180456462
1.8819872788950753
5027
2.026445679818972
1.93550795692971
4980
2.0011858902134483
1.5170738054966644
3994
1.9414893545285992
1.5490778103779308
5250
1.795437431995821
1.1230285427957079
5522
1.4307161072558676
1.1912218688703045
4905
1.178919865648366
1.0052738283697509
5058
1.1335701851765816
1.2563377521113075
5053
1.119376422181645
1.351682392419415
5128
1.0364085083790793
0.7646626152624142
4704
0.9534805926332195
0.7091479394396915
4620
0.8187094333808738
0.7904741371833593
4644
0.7667124751456517
0.6717414668842738
4509
0.7151571299524455
0.7534711441094563
4901
0.6859824065560081
0.6548699173047006
4231
0.6510991473529893
0.07250271251950834
5136
0.6240358212192276
0.7126815476299461
5044
0.56583626904628